# Define lists of tiles to be processed in the gridded exposure step
- `WITHELEV`: Include information on exposure, elevation, and other boundaries.
- `WITHOUTELEV`: Include information on exposure and other boundaries, but not elevation.
- `CIAM`: Include information on elevation and other boundaries, but not exposure

In [ ]:
from pathlib import Path

import dask.distributed as dd
import geopandas as gpd
import numpy as np
import pandas as pd
import rhg_compute_tools.kubernetes as rhgk
import xarray as xr
from shapely.geometry import box
from shapely.ops import unary_union

from sliiders import settings as sset
from sliiders import spatial

In [ ]:
nworkers = 16

client, cluster = rhgk.get_micro_cluster()

cluster.scale(nworkers)
cluster

In [ ]:
lat_size = 43200
lon_size = 86400

lats_per_deg, lons_per_deg = int(lat_size / 180), int(lon_size / 360)

lon_chunk = int(lon_size / nworkers)

In [ ]:
bdem = xr.open_dataarray(sset.PATH_SRTM15_PLUS, chunks={"lat": lats_per_deg}).persist()

bdem_max = (
    bdem.coarsen(lat=lats_per_deg).max().coarsen(lon=lons_per_deg).max().compute()
)

bdem_min = (
    bdem.coarsen(lat=lats_per_deg).min().coarsen(lon=lons_per_deg).min().compute()
)

bdem_max.plot()

In [ ]:
bdem_min.plot()

Double-check that the grid's spacing is regular over 1-degree tiles

In [ ]:
assert len(np.unique(np.floor(bdem.lat.values[:lats_per_deg]))) == 1
assert len(np.unique(np.floor(bdem.lon.values[:lons_per_deg]))) == 1

assert (np.floor(bdem.lat.values)[::lats_per_deg] == np.arange(-90, 90)).sum() == 180
assert (np.floor(bdem.lon.values)[::lons_per_deg] == np.arange(-180, 180)).sum() == 360

Shut down cluster

In [ ]:
client.close()
cluster.close()

## Organize tiles

In [ ]:
max_tiles = bdem_max.to_dataframe(name="max").reset_index()
min_tiles = bdem_min.to_dataframe(name="min").reset_index()

tiles = pd.merge(max_tiles, min_tiles, on=["lat", "lon"])

In [ ]:
def get_degree_box(row):
    """
    Get a 1-degree box containing a centroid
    defined by row["lon"] and row["lat"]
    """
    return box(
        row["lon"] - 0.5,
        row["lat"] - 0.5,
        row["lon"] + 0.5,
        row["lat"] + 0.5,
    )


def get_tile_names(df, lon_col, lat_col):
    """
    Get tile names in the format used by CoastalDEM.
    Defined by the southeastern point's 2-digit degree-distance
    north (N) or south (S) of the equator, and then its 3-digit
    distance east (E) or west (W) of the prime meridian.
    """
    tlon = np.floor(df[lon_col]).astype(int)
    tlat = np.floor(df[lat_col]).astype(int)

    NS = np.where(tlat >= 0, "N", "S")
    EW = np.where(tlon >= 0, "E", "W")

    return (
        NS
        + np.abs(tlat).astype(int).astype(str).str.zfill(2)
        + EW
        + np.abs(tlon).astype(int).astype(str).str.zfill(3)
    )


def get_all_exp_tiles(exp_path, filter_field=None):
    """
    Get the list of tiles included in an exposure dataset.
    """

    pq_filter = None
    if filter_field is not None:
        pq_filter = [[(filter_field, ">", 0)]]

    exp = pd.read_parquet(exp_path, columns=["x_ix", "y_ix"], filters=pq_filter)

    out = spatial.grid_ix_to_val(
        np.stack((exp["x_ix"], exp["y_ix"])).T, cell_size=sset.LITPOP_GRID_WIDTH
    )

    exp = exp.drop(columns=["x_ix", "y_ix"])
    exp["lon"] = np.floor(out[:, 0]).astype(int)
    exp["lat"] = np.floor(out[:, 1]).astype(int)

    lonlats = exp.drop_duplicates(["lon", "lat"]).reset_index(drop=True)

    lonlats["londir"] = np.where(lonlats["lon"] < 0, "W", "E")
    lonlats["latdir"] = np.where(lonlats["lat"] < 0, "S", "N")

    lonlats["lonnum"] = np.abs(lonlats["lon"]).astype(int).astype(str).str.zfill(3)
    lonlats["latnum"] = np.abs(lonlats["lat"]).astype(int).astype(str).str.zfill(2)

    lonlats["tile_name"] = (
        lonlats["latdir"] + lonlats["latnum"] + lonlats["londir"] + lonlats["lonnum"]
    )

    return lonlats["tile_name"].values

#### Determine whether each tile meets certain criteria, which will be used to define categories

Prepare tiles and category sets

In [ ]:
tiles["tile_name"] = get_tile_names(tiles, "lon", "lat")

tiles = gpd.GeoDataFrame(tiles, geometry=tiles.apply(get_degree_box, axis=1))

tiles["llat"] = np.floor(tiles["lat"])
tiles["llon"] = np.floor(tiles["lon"])

exp_tiles = get_all_exp_tiles(sset.PATH_EXPOSURE_BLENDED, filter_field="value")
pop_tiles = get_all_exp_tiles(sset.PATH_LANDSCAN_INT, filter_field="population")

coastaldem_tiles = [t.stem for t in sset.DIR_COASTALDEM.glob("*.tif")]

Apply category logic

In [ ]:
# Tile is included in CoastalDEM
tiles["coastaldem"] = tiles["tile_name"].isin(coastaldem_tiles)

# Tile has non-0 asset-value
tiles["exp"] = tiles["tile_name"].isin(exp_tiles)

# Tile has non-0 population
tiles["pop"] = tiles["tile_name"].isin(pop_tiles)

# Tile is below the 60th parallel south (governed under the Antarctic Treaty System)
tiles["antarctica"] = tiles["lat"] < -60

# Tile includes elevations below 50 meters
tiles["below50"] = tiles["min"] <= 50

# Tile includes elevations above -50 meters
tiles["above_neg50"] = tiles["max"] >= -50

Save list of low-lying tiles that are not contiguous with the ocean ("inland")

In [ ]:
sset.DIR_EXPOSURE_BINNED

In [ ]:
ocean_shape = tiles[tiles["below50"]].buffer(0.01).unary_union

ocean_shape = list(ocean_shape.geoms)[np.argmax([g.area for g in ocean_shape.geoms])]

tiles["contiguous_with_ocean"] = tiles["geometry"].within(ocean_shape)

tiles[tiles["contiguous_with_ocean"]].plot(figsize=(20, 20))

inland = (
    tiles[(tiles["coastaldem"]) & (~tiles["contiguous_with_ocean"])][["tile_name"]]
    .sort_values("tile_name")
    .reset_index(drop=True)
)
inland.to_parquet(sset.PATH_INLAND_TILE_LIST, index=False)

Categorize tiles based on whether they are relevant to each group

In [ ]:
tiles = tiles[~tiles["tile_name"].isin(inland["tile_name"].to_numpy())].reset_index(
    drop=True
)

tiles["WITHELEV"] = (tiles["below50"] | tiles["coastaldem"]) & tiles["exp"]
tiles["WITHOUTELEV"] = tiles["exp"] & (~tiles["WITHELEV"])
tiles["CIAM"] = (
    (tiles["above_neg50"])
    & (tiles["below50"] | tiles["coastaldem"])
    & (~tiles["antarctica"])
    & (~tiles["exp"])
)

Plot tile categories

In [ ]:
def get_color(tile):
    if tile["WITHELEV"]:
        return "purple"
    if tile["WITHOUTELEV"]:
        return "green"
    if tile["CIAM"]:
        return "orange"
    return "blue"


tiles["color"] = tiles.apply(get_color, axis=1)
tiles.plot(color=tiles["color"], figsize=(20, 20))

Transform booleans into categories

In [ ]:
tiles["PROCESSING_SET"] = np.where(
    tiles["WITHELEV"],
    "WITHELEV",
    np.where(
        tiles["WITHOUTELEV"], "WITHOUTELEV", np.where(tiles["CIAM"], "CIAM", None)
    ),
)

Save lists

In [ ]:
out = tiles[["tile_name", "PROCESSING_SET"]]

out = out[pd.notnull(out["PROCESSING_SET"])].reset_index(drop=True)

In [ ]:
out.to_parquet(sset.PATH_EXPOSURE_TILE_LIST, index=False)